Course work

Author: sushmaakoju

## Baseline LLM4Judge challenge

### Configuring the baseline

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install jsonlines
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 10.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-

In [3]:
!git clone https://github.com/llm4eval/LLMJudge.git

Cloning into 'LLMJudge'...
remote: Enumerating objects: 34, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 34 (delta 7), reused 14 (delta 2), pack-reused 0
Receiving objects: 100% (34/34), 2.33 MiB | 18.61 MiB/s, done.
Resolving deltas: 100% (7/7), done.


In [4]:
!pip install --upgrade huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.7/401.7 kB 10.2 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.23.2
    Uninstalling huggingface-hub-0.23.2:
      Successfully uninstalled huggingface-hub-0.23.2


In [5]:
!huggingface-cli logout
!huggingface-cli login

Not logged in!

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
!huggingface-cli whoami

In [8]:
!huggingface-cli download meta-llama/Meta-Llama-3-8B-Instruct --include "original/*" --local-dir Meta-Llama-3-8B-Instruct

Fetching 3 files:   0% 0/3 [00:00<?, ?it/s]Downloading 'original/tokenizer.model' to 'Meta-Llama-3-8B-Instruct/.huggingface/download/original/tokenizer.model.82e9d31979e92ab929cd544440f129d9ecd797b69e327f80f17e1c50d5551b55.incomplete'

original/params.json: 100% 211/211 [00:00<00:00, 1.09MB/s]
Download complete. Moving file to Meta-Llama-3-8B-Instruct/original/params.json

consolidated.00.pth:   0% 0.00/16.1G [00:00<?, ?B/s]

tokenizer.model: 100% 2.18M/2.18M [00:00<00:00, 65.4MB/s]
Download complete. Moving file to Meta-Llama-3-8B-Instruct/original/tokenizer.model

consolidated.00.pth:   0% 21.0M/16.1G [00:00<01:52, 142MB/s]
consolidated.00.pth:   0% 52.4M/16.1G [00:00<01:14, 215MB/s]
consolidated.00.pth:   1% 83.9M/16.1G [00:00<01:03, 253MB/s]
consolidated.00.pth:   1% 126M/16.1G [00:00<01:07, 238MB/s] 
consolidated.00.pth:   1% 157M/16.1G [00:00<01:20, 197MB/s]
consolidated.00.pth:   1% 189M/16.1G [00:00<01:24, 188MB/s]
consolidated.00.pth:   1% 220M/16.1G [00:01<01:20, 197MB/s]
con

In [8]:
#!huggingface-cli login --token $HF_TOKEN

In [9]:
import pandas as pd
import jsonlines
from tqdm import tqdm

import transformers
import torch

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [10]:
torch.set_default_device("cuda")

In [11]:
docid_to_doc = dict()

with jsonlines.open('/content/LLMJudge/data/llm4eval_document_2024.jsonl', 'r') as document_file:
  for obj in document_file:
    docid_to_doc[obj['docid']] = obj['doc']

In [12]:
query_data = pd.read_csv("/content/LLMJudge/data/llm4eval_query_2024.txt", sep="\t", header=None, names=['qid', 'qtext'])
qid_to_query = dict(zip(query_data.qid, query_data.qtext))

In [13]:
system_message = """You are a search quality rater evaluating the relevance of passages. Given a query and passage, you must provide a score on an integer scale of 0 to 3 with the following meanings:

    3 = Perfectly relevant: The passage is dedicated to the query and contains the exact answer.
    2 = Highly relevant: The passage has some answer for the query, but the answer may be a bit unclear, or hidden amongst extraneous information.
    1 = Related: The passage seems related to the query but does not answer it.
    0 = Irrelevant: The passage has nothing to do with the query

    Assume that you are writing an answer to the query. If the passage seems to be related to the query but does not include any answer to the query, mark it 1. If you would use any of the information contained in the passage in such an asnwer, mark it 2. If the passage is primarily about the query, or contains vital information about the topic, mark it 3. Otherwise, mark it 0."""


In [14]:

def get_prompt(query, passage):
    return f"""Please rate how the given passage is relevant to the query. The output must be only a score that indicate how relevant they are.

    Query: {query}
    Passage: {passage}

    Score:"""

In [15]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [16]:
def get_relevance_score(prompt):
  messages = [
      {"role": "system", "content": system_message},
      {"role": "user", "content": prompt},
  ]

  prompt = pipeline.tokenizer.apply_chat_template(
          messages,
          tokenize=False,
          add_generation_prompt=True
  )

  terminators = [
      pipeline.tokenizer.eos_token_id,
      pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
  ]

  outputs = pipeline(
      prompt,
      max_new_tokens=256,
      eos_token_id=terminators,
      pad_token_id=128009,
      do_sample=True,
      temperature=0.6,
      top_p=0.9,
  )

  return outputs[0]["generated_text"][len(prompt):]

In [17]:
test_qrel = pd.read_csv("/content/LLMJudge/data/llm4eval_test_qrel_2024.txt", sep=" ", header=None, names=['qid', 'Q0', 'docid'])
test_qrel.head(5)

,qid,Q0,docid
0,q49,0,p3659
1,q49,0,p11027
2,q49,0,p1270
3,q49,0,p786
4,q49,0,p9577


In [ ]:
with open('llm4eval_test_qrel_results.txt', 'w') as result_file:
  for eachline in tqdm(test_qrel.itertuples(index=True)):
    qidx = eachline.qid
    docidx = eachline.docid
    prompt = get_prompt(query=qid_to_query[qidx], passage=docid_to_doc[docidx])
    pred_score = get_relevance_score(prompt)
    result_file.write(f"{qidx} 0 {docidx} {pred_score}\n")

10it [00:44,  4.12s/it]You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
128it [08:53,  4.20s/it]